In [12]:
import cv2
from detectors import FaceDetector
from classifiers import MultiLayerPerceptron
#import matplotlib.pyplot as plt
from IPython.display import display, HTML
import base64
import homebrew
import numpy as np

In [2]:
def imshow(name, imageArray):
     _, png = cv2.imencode('.png', imageArray)
     encoded = base64.b64encode(png)
     return HTML(data='''<img alt="{0}" src="data:image/png;base64, {1}"/>'''.format(name, encoded.decode('ascii')))

In [3]:
samples = []
labels = []
image = cv2.imread('/home/vs/ENAS/cv2/jaffe/ml-face-jaffe-dataset/dataset/MK.DI1.128.tiff', cv2.IMREAD_COLOR | cv2.IMREAD_ANYDEPTH)

imshow('MK.DT1',image)

In [17]:
face_casc='params/haarcascade_frontalface_default.xml'
left_eye_casc='params/haarcascade_lefteye_2splits.xml'
right_eye_casc='params/haarcascade_righteye_2splits.xml'
faces = FaceDetector(face_casc, left_eye_casc, right_eye_casc)
image = cv2.imread('/home/vs/ENAS/cv2/jaffe/ml-face-jaffe-dataset/dataset/YM.SA2.56.tiff', cv2.IMREAD_COLOR | cv2.IMREAD_ANYDEPTH)
success, frame, head = faces.detect(image)
imshow('MK.DI2',frame)

In [5]:
image = cv2.imread('/home/vs/ENAS/cv2/jaffe/ml-face-jaffe-dataset/dataset/KL.SA3.163.tiff', cv2.IMREAD_COLOR | cv2.IMREAD_ANYDEPTH)
success, frame, head = faces.detect(image)
success, head  = faces.align_head(head)
imshow('KM.SA2.10.tiff',head)

In [6]:
import glob
face_casc='params/haarcascade_frontalface_default.xml'
left_eye_casc='params/haarcascade_lefteye_2splits.xml'
right_eye_casc='params/haarcascade_righteye_2splits.xml'
faces = FaceDetector(face_casc, left_eye_casc, right_eye_casc)
filenames = glob.glob('dataset/*.tiff')
#filenames
#f_name = [o.split('/')[-1] for o in filenames]
for f in filenames:
    #read image
    image = cv2.imread(f, cv2.IMREAD_COLOR | cv2.IMREAD_ANYDEPTH)
    #get the label
    f_name = f.split('/')[-1]
    print(f_name)
    label = f_name[3:5]
    success, frame, head = faces.detect(image)
    #height, width = head.shape[:2]
    #print(0.2 * height)
    #print(0.5 * width)
    #left_eye_region = head[int(0.2*height):int(0.5*height), int(0.1*width):int(0.5*width)]
    #print(left_eye_region.shape)
    success, head  = faces.align_head(head)
    samples.append(head.flatten())
    labels.append(label)

   


TM.FE3.198.tiff
KL.SA3.163.tiff
YM.SA2.56.tiff
NM.FE1.110.tiff
NM.SA1.98.tiff
KR.NE3.73.tiff
KR.AN2.84.tiff
UY.SU3.145.tiff
KA.NE2.27.tiff
KA.AN2.40.tiff
NM.AN1.104.tiff
KL.NE2.156.tiff
KR.DI2.87.tiff
YM.HA2.53.tiff
UY.SU2.144.tiff
TM.SU3.189.tiff
TM.SU1.187.tiff
KA.HA4.32.tiff
KL.AN1.167.tiff
KR.FE3.91.tiff
KM.FE3.25.tiff
YM.AN3.63.tiff
KR.HA2.75.tiff
KL.SU1.164.tiff
KA.SA1.33.tiff
NM.SA3.100.tiff
UY.DI3.151.tiff
MK.NE2.114.tiff
TM.AN2.191.tiff
YM.AN1.61.tiff
KM.SA5.13.tiff
KM.FE1.23.tiff
NM.DI3.109.tiff
UY.NE3.136.tiff
YM.HA3.54.tiff
KR.SU2.81.tiff
MK.DI2.129.tiff
KR.AN1.83.tiff
KA.FE4.48.tiff
KL.DI4.173.tiff
KM.FE2.24.tiff
TM.AN1.190.tiff
MK.DI3.130.tiff
NA.AN2.212.tiff
UY.NE2.135.tiff
UY.FE1.152.tiff
UY.HA1.137.tiff
YM.NE3.51.tiff
MK.SU2.123.tiff
TM.SA2.185.tiff
NA.SA1.205.tiff
NA.FE2.218.tiff
KM.DI3.22.tiff
KR.FE1.89.tiff
TM.DI1.193.tiff
TM.NE1.177.tiff
YM.SU1.58.tiff
MK.SA1.119.tiff
KA.HA3.31.tiff
TM.DI3.195.tiff
NA.DI3.216.tiff
KR.SU1.80.tiff
KR.SA3.79.tiff
YM.SA1.55.tiff
KA.SU2

In [7]:
import pickle
data_file = 'faces_training.pkl'
#save samples data and labels
f = open(data_file, 'wb')
pickle.dump(samples, f)
pickle.dump(labels, f)
f.close()

In [15]:
 # load training data
    # training data can be recorded using chapter7.py in training mode
(X_train, y_train), (X_test, y_test), _, _ = homebrew.load_data(
    "faces_training.pkl",
    num_components=50,
    test_split=0.2,
    save_to_file="faces_preprocessed.pkl",
    seed=42)
if len(X_train) == 0 or len(X_test) == 0:
    print "Empty data"
    #raise SystemExit
    
# convert to numpy
X_train = np.squeeze(np.array(X_train)).astype(np.float32)
y_train = np.array(y_train)
X_test = np.squeeze(np.array(X_test)).astype(np.float32)
y_test = np.array(y_test)

# find all class labels
labels = np.unique(np.hstack((y_train, y_test)))

# prepare training
num_features = len(X_train[0])
num_classes = len(labels)
params = dict(term_crit=(cv2.TERM_CRITERIA_COUNT, 300, 0.01),
                  train_method=cv2.ANN_MLP_TRAIN_PARAMS_BACKPROP,
                  bp_dw_scale=0.001, bp_moment_scale=0.9)
saveFile = 'params/mlp.xml'

# find best MLP configuration
print "---"
print "1-hidden layer networks"
best_acc = 0.0  # keep track of best accuracy
for l1 in xrange(10):
    # gradually increase the hidden-layer size
    layerSizes = np.int32([num_features, (l1 + 1) * num_features/5,
                               num_classes])
    MLP = MultiLayerPerceptron(layerSizes, labels)
    print layerSizes
    MLP.fit(X_train, y_train, params=params)
    (acc, _, _) = MLP.evaluate(X_train, y_train)
    print " - train acc = ", acc
    (acc, _, _) = MLP.evaluate(X_test, y_test)
    print " - test acc = ", acc
    if acc > best_acc:
    # save best MLP configuration to file
        MLP.save(saveFile)
        best_acc = acc




    

Loading data from faces_training.pkl
Loaded 213 training samples
Save preprocessed data to faces_preprocessed.pkl
---
1-hidden layer networks
[50 10  7]
 - train acc =  0.952941176471
 - test acc =  0.627906976744
[50 20  7]
 - train acc =  0.964705882353
 - test acc =  0.558139534884
[50 30  7]
 - train acc =  0.964705882353
 - test acc =  0.53488372093
[50 40  7]
 - train acc =  0.976470588235
 - test acc =  0.581395348837
[50 50  7]
 - train acc =  0.976470588235
 - test acc =  0.511627906977
[50 60  7]
 - train acc =  0.976470588235
 - test acc =  0.558139534884
[50 70  7]
 - train acc =  0.982352941176
 - test acc =  0.604651162791
[50 80  7]
 - train acc =  0.970588235294
 - test acc =  0.46511627907
[50 90  7]
 - train acc =  0.976470588235
 - test acc =  0.46511627907
[ 50 100   7]
 - train acc =  0.976470588235
 - test acc =  0.511627906977


In [16]:
from os import path 
load_preprocessed_data = 'faces_preprocessed.pkl'
load_mlp='params/mlp.xml'
if path.isfile(load_preprocessed_data):
    (X_train, y_train), (X_test, y_test), V, m = homebrew.load_from_file(load_preprocessed_data)
    pca_V = V
    pca_m = m
    all_labels = np.unique(np.hstack((y_train, y_test)))
    layer_sizes = np.array([pca_V.shape[1], len(all_labels)])
    MLP = MultiLayerPerceptron(layer_sizes, all_labels)
    if path.isfile(load_mlp):
        MLP.load(load_mlp)

In [28]:
image = cv2.imread('/home/vs/ENAS/cv2/jaffe/ml-face-jaffe-dataset/dataset/MK.SU3.124.tiff', cv2.IMREAD_COLOR | cv2.IMREAD_ANYDEPTH)
success, frame, head = faces.detect(image)
success, head  = faces.align_head(head)
X, _, _ = homebrew.extract_features([head.flatten()],pca_V, pca_m)
 # predict label with pre-trained MLP
label = MLP.predict(np.array(X))[0]
# draw label above bounding box
cv2.putText(frame, str(label), (220, 200 - 20), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
imshow('prediction', frame)